In [3]:
%matplotlib inline
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [8]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True,reshape=False)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [10]:
mnist.train.images.shape # 个数 长 宽 通道数

(55000, 28, 28, 1)

In [26]:
with tf.Graph().as_default() as g:
    inputs = tf.placeholder(shape=[None, 28, 28, 1], dtype=tf.float32)
    labels = tf.placeholder(shape=[None,10],dtype=tf.float32)
    
    conv1_weight = tf.get_variable('c1_w',[5,5,1,8],dtype=tf.float32)
    conv1_bias = tf.get_variable('c1_b',[8],dtype=tf.float32)
    conv1 = tf.nn.conv2d(inputs, conv1_weight, [1,1,1,1],'VALID')
    conv1 = tf.nn.relu(conv1 + conv1_bias) # shape[None,24,24,8]
    
    pool1 = tf.nn.max_pool(conv1,[1, 2, 2, 1],[1, 2, 2, 1],'SAME')# shape =[None,12,12,8]
    conv2_weight = tf.get_variable('c2_w',shape=[5, 5, 8, 16],dtype=tf.float32)
    conv2_bias = tf.get_variable('c2_b',shape=[16],dtype=tf.float32)
    conv2 = tf.nn.conv2d(pool1, conv2_weight,[1,1,1,1],'VALID')
    conv2 = tf.nn.relu(conv2 + conv2_bias)#shape=[None,8,8,16]
    
    pool2 = tf.nn.max_pool(conv2,[1, 2, 2, 1],[1, 2, 2, 1],'SAME')
    
    #pool2_shape = tf.reshape(pool2)
    tmp = tf.reshape(pool2,[-1, 4 * 4 * 16])
    #tmp = tf.reshape(pool2,[-1,],pool2_shape[1]*pool2_shape[2]*pool2_shape[3])
    logits = tf.keras.layers.Dense(10,activation=None)(tmp)
    output = tf.nn.softmax(logits)
    
    #loss=tf.reduce_mean(tf.keras.losses.categorical_crossentropy(labels,logits))
    loss = tf.reduce_mean(-1 * tf.reduce_sum(labels * tf.log(output),axis=1))
    acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(labels,axis=1),tf.argmax(output,axis=1)),tf.float32))

with tf.Session (graph=g) as sess:
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.001) #梯度下降训练 比较原始 学习率
    train_op = optim.minimize(loss) #最小化代价
    sess.run(tf.global_variables_initializer())
    
    for i in range(5000):
        batch_img,batch_labels = mnist.train.next_batch(32)
        res_loss, _ = sess.run([loss, train_op],feed_dict={inputs: batch_img,
                                                           labels:batch_labels})
    for step in range(20000):
        batch_images, batch_labels = mnist.train.next_batch(32)
        res_loss, _ = sess.run([loss, train_op], feed_dict={
            inputs: batch_images,
            labels: batch_labels
        })
        
        # 输出代价并验证模型
        if step % 500 == 0:
            accs = []
            for test_step in range(10000 // 32):
                batch_images, batch_labels = mnist.test.next_batch(32)
                res_acc = sess.run(acc, feed_dict={
                    inputs: batch_images,
                    labels: batch_labels
                })
                accs.append(res_acc)
            accs = np.mean(accs)
            print('step %5d, loss %2.4f, acc %.4f' % (step, res_loss, accs))
            #saver.save(sess, 'conv_ckpt')    
#         if i % 500 == 0:
#             print(res_loss)
#             batch_img, _ = mnist.test.next_batch(5)
#             res_out = sess.run(out, feed_dict={feed_dict={inputs: batch_img,
#                                                           labels:batch_labels})
#             for j in range(10):
#                 plt.subplot(2, 5, j + 1)
#                 img = batch_img[j] if j < 5 else res_out[j - 5]
#                 img = (img *255).astype(np.uint8).reshape([28, 28])
#                 plt.imshow(img)
#             plt.show()

step     0, loss 0.6364, acc 0.8501
step   500, loss 0.4860, acc 0.8647
step  1000, loss 0.4887, acc 0.8750
step  1500, loss 0.4145, acc 0.8829
step  2000, loss 0.2705, acc 0.8913
step  2500, loss 0.3114, acc 0.8946
step  3000, loss 0.4047, acc 0.9008
step  3500, loss 0.3144, acc 0.9035
step  4000, loss 0.2273, acc 0.9068
step  4500, loss 0.3413, acc 0.9073
step  5000, loss 0.5139, acc 0.9132
step  5500, loss 0.1702, acc 0.9157
step  6000, loss 0.3118, acc 0.9180
step  6500, loss 0.2688, acc 0.9195
step  7000, loss 0.2750, acc 0.9216
step  7500, loss 0.3613, acc 0.9246
step  8000, loss 0.3477, acc 0.9266
step  8500, loss 0.4376, acc 0.9276
step  9000, loss 0.1360, acc 0.9290
step  9500, loss 0.2537, acc 0.9296
step 10000, loss 0.1691, acc 0.9323
step 10500, loss 0.2618, acc 0.9354
step 11000, loss 0.0880, acc 0.9357
step 11500, loss 0.1450, acc 0.9339
step 12000, loss 0.2287, acc 0.9376
step 12500, loss 0.1479, acc 0.9402
step 13000, loss 0.4217, acc 0.9419
step 13500, loss 0.3361, acc